<a href="https://colab.research.google.com/github/DaeSeokSong/MachineLearningModels/blob/main/BlackUP_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive Local Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Main Code

In [ ]:
%cd drive/MyDrive/DeepLearning/BlackUp/Dataset

studyData = pd.read_csv("KOSPI200_20y_dataset.csv");
testData = pd.read_csv("005930_10y_dataset.csv")

closePrice = studyData["Close"].values
openPrice = studyData["Open"].values
highPrice = studyData["High"].values
lowPrice = studyData["Low"].values
tradingVolume = studyData["Trading"].values
chageRate = studyData["ChageRate"].values

# keras.models 중 모델을 순차적으로 정의하는 클래스
model = Sequential()

model.add(LSTM(
    1, # 해당 층의 노드 개수
    input_shape=(50,1), # input_shape=입력값 모양
    return_sequences=True)) # return_sequences == 각 시퀀스를 출력할지
model.add(Dropout(0.2)) # 과적합 방지용 Dropout 20%(==0.2)

model.add(LSTM(
    100, # 노드 100개
    return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(1, activation='sigmoid')) # 활성화(Activation)함수 = sigmoid
model.compile(loss='mse', optimizer='rmsprop')

model.summary()